# Alpaca-py crypto trading basic

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/crypto-trading-basic.ipynb)

- This notebook shows how to use alpaca-py with crypto trading API endpoints
- Please use ``paper account``. Please ``DO NOT`` use this notebook with live account. In this notebook, we place orders for crypto as an example.

In [1]:
# Please change the following to your own PAPER api key and secret
# You can get them from https://alpaca.markets/
import credentials as cs
api_key = cs.api_key
secret_key = cs.secret_key

#### We use paper environment for this example ####
paper = True # Please do not modify this. This example is for paper trading only.
####

# Below are the variables for development this documents
# Please do not change these variables
trade_api_url = None
trade_api_wss = None
data_api_url = None
stream_data_wss = None

In [ ]:
# install alpaca-py
! python3 -m pip install alpaca-py

In [2]:
import json
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

import alpaca
from alpaca.data.live.crypto import *
from alpaca.data.historical.crypto import *
from alpaca.data.requests import *
from alpaca.data.timeframe import *
from alpaca.trading.client import *
from alpaca.trading.stream import *
from alpaca.trading.requests import *
from alpaca.trading.enums import *
from alpaca.common.exceptions import APIError

In [3]:
# to run async code in jupyter notebook
import nest_asyncio
nest_asyncio.apply()

In [4]:
# check version of alpaca-py
alpaca.__version__

'0.23.0'

# Trading Client

In [5]:
# setup clients
trade_client = TradingClient(api_key=api_key, secret_key=secret_key, paper=paper, url_override=trade_api_url)

In [6]:
# check trading account
# You can check definition of each field in the following documents
# ref. https://docs.alpaca.markets/docs/account-plans
# ref. https://docs.alpaca.markets/reference/getaccount-1
acct = trade_client.get_account()
acct

{   'account_blocked': False,
    'account_number': 'PA3JL9EDDPLL',
    'accrued_fees': '0',
    'buying_power': '199998',
    'cash': '99999',
    'created_at': datetime.datetime(2023, 5, 26, 10, 24, 2, 366956, tzinfo=TzInfo(UTC)),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '99999',
    'id': UUID('09818092-d381-4bcf-a941-8f1ec2ae82c0'),
    'initial_margin': '0',
    'last_equity': '99999',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '99999',
    'options_approved_level': 2,
    'options_buying_power': '99999',
    'options_trading_level': 2,
    'pattern_day_trader': True,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '99999',
    'regt_buying_power': '199998',
    'short_market_value': '0',
    'shorting_enabled': True,
  

In [ ]:
# check account configuration
# ref. https://docs.alpaca.markets/reference/getaccountconfig-1
acct_config = trade_client.get_account_configurations()
acct_config

In [ ]:
# get list of crypto pairs
# ref. https://docs.alpaca.markets/reference/get-v2-assets-1
req = GetAssetsRequest(
  asset_class=AssetClass.CRYPTO,
  status=AssetStatus.ACTIVE
)
assets = trade_client.get_all_assets(req)
assets[:2]

### Orders

In [7]:
# we will place orders which Alapca trading platform supports
# - order types for crypto: market, limit, stop_limit
# - time_in_force for crypto: gtc, ioc.
# please refer to the following documents for more details
# ref. https://docs.alpaca.markets/docs/orders-at-alpaca
# ref. https://docs.alpaca.markets/reference/postorder
trail

# we will place orders for symbol: BTC/USD in this example
symbol = "BTC/USD"

In [8]:
# simple, market order
# you can specify:
# or notional value (e.g. 100 USD) (which is in the next example)
req = MarketOrderRequest(
    symbol = symbol,
    qty = 0.01,
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '4a861c58-5b79-4a1c-987b-067e23077f93',
    'created_at': datetime.datetime(2024, 6, 15, 14, 37, 17, 644728, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('7df2d9bc-052f-4062-aeee-2c1dd7c25880'),
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 6, 15, 14, 37, 17, 644728, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD'

In [9]:
# simple, market order, notional

req = MarketOrderRequest(
    symbol = symbol,
    notional = 1.11,  # notional is specified in USD, here we specify $1.11
    side = OrderSide.BUY,
    type = OrderType.MARKET,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': 'ec77104d-2a82-4828-8da2-5ce7b6f4fe4f',
    'created_at': datetime.datetime(2024, 6, 15, 14, 38, 39, 175963, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('539876c9-14df-4ea3-8e36-da8ef9de6c66'),
    'legs': None,
    'limit_price': None,
    'notional': '1.11',
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.MARKET: 'market'>,
    'qty': None,
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 6, 15, 14, 38, 39, 175963, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD'

In [15]:
# simple, limit order
req = LimitOrderRequest(
    symbol = symbol,
    qty = 0.01,
    limit_price = 60000,
    side = OrderSide.BUY,
    type = OrderType.LIMIT,
    time_in_force = TimeInForce.GTC,
)
res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '63c3e2fb-9037-4c38-bfbb-16ca0e3e7420',
    'created_at': datetime.datetime(2024, 6, 15, 14, 44, 3, 377101, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('69de61c6-337d-4f85-b614-06fee1f54c68'),
    'legs': None,
    'limit_price': '60000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.LIMIT: 'limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.PENDING_NEW: 'pending_new'>,
    'stop_price': None,
    'submitted_at': datetime.datetime(2024, 6, 15, 14, 44, 3, 377101, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD',

In [14]:
# stop limit order
req = StopLimitOrderRequest(
                    symbol = symbol,
                    qty = 0.01,
                    side = OrderSide.BUY,
                    time_in_force = TimeInForce.GTC,
                    limit_price = 55000,
                    stop_price = 60000
                    )

res = trade_client.submit_order(req)
res

{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
    'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
    'canceled_at': None,
    'client_order_id': '2f37176c-3abd-45cb-abbd-e405b8049ae2',
    'created_at': datetime.datetime(2024, 6, 15, 14, 43, 59, 64905, tzinfo=TzInfo(UTC)),
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': UUID('942a9675-9d8a-444a-9188-8c008e80ebee'),
    'legs': None,
    'limit_price': '55000',
    'notional': None,
    'order_class': <OrderClass.SIMPLE: 'simple'>,
    'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
    'qty': '0.01',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': <OrderSide.BUY: 'buy'>,
    'status': <OrderStatus.NEW: 'new'>,
    'stop_price': '60000',
    'submitted_at': datetime.datetime(2024, 6, 15, 14, 43, 59, 64905, tzinfo=TzInfo(UTC)),
    'symbol': 'BTC/USD',
  

In [12]:
# get a list of orders including closed (e.g. filled) orders by specifying symbol
req = GetOrdersRequest(
    status = QueryOrderStatus.ALL,
    symbols = [symbol]
)
orders = trade_client.get_orders(req)
orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': datetime.datetime(2024, 6, 15, 14, 42, 24, 163546, tzinfo=TzInfo(UTC)),
     'client_order_id': '2b5761c0-285e-40e9-be45-cb57be2ccb57',
     'created_at': datetime.datetime(2024, 6, 15, 14, 41, 28, 4789, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('9242e34a-913f-4c83-9a98-5b87cb5bc613'),
     'legs': None,
     'limit_price': '55000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.STOP_LIMIT: 'stop_limit'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.CANCELED: 'canceled'>,
     'stop_price': '60000',
     'submitted_at

In [13]:
import pandas as pd
df=pd.DataFrame(orders)
df

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,"(id, 9242e34a-913f-4c83-9a98-5b87cb5bc613)","(client_order_id, 2b5761c0-285e-40e9-be45-cb57...","(created_at, 2024-06-15 14:41:28.004789+00:00)","(updated_at, 2024-06-15 14:42:24.176585+00:00)","(submitted_at, 2024-06-15 14:42:09.599089+00:00)","(filled_at, None)","(expired_at, None)","(canceled_at, 2024-06-15 14:42:24.163546+00:00)","(failed_at, None)","(replaced_at, None)",...,"(side, OrderSide.BUY)","(time_in_force, TimeInForce.GTC)","(limit_price, 55000)","(stop_price, 60000)","(status, OrderStatus.CANCELED)","(extended_hours, False)","(legs, None)","(trail_percent, None)","(trail_price, None)","(hwm, None)"
1,"(id, 685328c3-3778-4c28-8007-1dd7445b6973)","(client_order_id, ab81850b-5bb6-4a4b-a7ef-a1a5...","(created_at, 2024-06-15 14:39:54.942233+00:00)","(updated_at, 2024-06-15 14:40:49.927170+00:00)","(submitted_at, 2024-06-15 14:39:54.942233+00:00)","(filled_at, None)","(expired_at, None)","(canceled_at, 2024-06-15 14:40:49.874121+00:00)","(failed_at, None)","(replaced_at, None)",...,"(side, OrderSide.BUY)","(time_in_force, TimeInForce.GTC)","(limit_price, 60000)","(stop_price, None)","(status, OrderStatus.CANCELED)","(extended_hours, False)","(legs, None)","(trail_percent, None)","(trail_price, None)","(hwm, None)"
2,"(id, 539876c9-14df-4ea3-8e36-da8ef9de6c66)","(client_order_id, ec77104d-2a82-4828-8da2-5ce7...","(created_at, 2024-06-15 14:38:39.175964+00:00)","(updated_at, 2024-06-15 14:38:39.227308+00:00)","(submitted_at, 2024-06-15 14:38:39.175964+00:00)","(filled_at, 2024-06-15 14:38:39.179322+00:00)","(expired_at, None)","(canceled_at, None)","(failed_at, None)","(replaced_at, None)",...,"(side, OrderSide.BUY)","(time_in_force, TimeInForce.GTC)","(limit_price, None)","(stop_price, None)","(status, OrderStatus.FILLED)","(extended_hours, False)","(legs, None)","(trail_percent, None)","(trail_price, None)","(hwm, None)"
3,"(id, 7df2d9bc-052f-4062-aeee-2c1dd7c25880)","(client_order_id, 4a861c58-5b79-4a1c-987b-067e...","(created_at, 2024-06-15 14:37:17.644729+00:00)","(updated_at, 2024-06-15 14:37:17.677332+00:00)","(submitted_at, 2024-06-15 14:37:17.644729+00:00)","(filled_at, 2024-06-15 14:37:17.647436+00:00)","(expired_at, None)","(canceled_at, None)","(failed_at, None)","(replaced_at, None)",...,"(side, OrderSide.BUY)","(time_in_force, TimeInForce.GTC)","(limit_price, None)","(stop_price, None)","(status, OrderStatus.FILLED)","(extended_hours, False)","(legs, None)","(trail_percent, None)","(trail_price, None)","(hwm, None)"


In [16]:
# see all open orders
req = GetOrdersRequest(
    status = QueryOrderStatus.OPEN,
    symbols = [symbol]
)
open_orders = trade_client.get_orders(req)
open_orders

[{   'asset_class': <AssetClass.CRYPTO: 'crypto'>,
     'asset_id': UUID('276e2673-764b-4ab6-a611-caf665ca6340'),
     'canceled_at': None,
     'client_order_id': '63c3e2fb-9037-4c38-bfbb-16ca0e3e7420',
     'created_at': datetime.datetime(2024, 6, 15, 14, 44, 3, 377101, tzinfo=TzInfo(UTC)),
     'expired_at': None,
     'extended_hours': False,
     'failed_at': None,
     'filled_at': None,
     'filled_avg_price': None,
     'filled_qty': '0',
     'hwm': None,
     'id': UUID('69de61c6-337d-4f85-b614-06fee1f54c68'),
     'legs': None,
     'limit_price': '60000',
     'notional': None,
     'order_class': <OrderClass.SIMPLE: 'simple'>,
     'order_type': <OrderType.LIMIT: 'limit'>,
     'qty': '0.01',
     'replaced_at': None,
     'replaced_by': None,
     'replaces': None,
     'side': <OrderSide.BUY: 'buy'>,
     'status': <OrderStatus.NEW: 'new'>,
     'stop_price': None,
     'submitted_at': datetime.datetime(2024, 6, 15, 14, 44, 3, 377101, tzinfo=TzInfo(UTC)),
     'symbol':

In [17]:
# cancel all open orders
trade_client.cancel_orders()

[{'id': UUID('69de61c6-337d-4f85-b614-06fee1f54c68'), 'status': 200},
 {'id': UUID('942a9675-9d8a-444a-9188-8c008e80ebee'), 'status': 200}]

### Positions

In [ ]:
# get all open positions
# ref. https://docs.alpaca.markets/reference/getallopenpositions-1
positions = trade_client.get_all_positions()
positions

In [ ]:
# get positions by symbol
# ref. https://docs.alpaca.markets/reference/getopenposition-1
position = trade_client.get_open_position(symbol_or_asset_id=symbol)
position


In [ ]:
# get positions by asset_id
trade_client.get_open_position(symbol_or_asset_id=position.asset_id)

In [ ]:
# close the position with specifying qty
# ref. https://docs.alpaca.markets/reference/deleteopenposition-1
trade_client.close_position(
    symbol_or_asset_id = symbol,
    close_options = ClosePositionRequest(
        qty = "0.01",
    )
)

# Trade Update (Stream)

With TradingStream client, you can get updates about trades

You can open this notebook in another window and run below cell to check trade updates.

In [ ]:
# subscribe trade updates
trade_stream_client = TradingStream(api_key, secret_key, paper=paper, url_override = trade_api_wss)

async def trade_updates_handler(data):
    print(data)

trade_stream_client.subscribe_trade_updates(trade_updates_handler)
trade_stream_client.run()

# Market Data (Historical)

In [ ]:
# setup crypto historical data client
crypto_historical_data_client = CryptoHistoricalDataClient()

In [ ]:
# get historical bars by symbol
# ref. https://docs.alpaca.markets/reference/cryptobars-1
now = datetime.now(ZoneInfo("America/New_York"))
req = CryptoBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 1, unit = TimeFrameUnit.Hour), # specify timeframe
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    limit = 2,                                               # specify limit
)
crypto_historical_data_client.get_crypto_bars(req).df

In [ ]:
# get historical trades by symbol
req = CryptoTradesRequest(
    symbol_or_symbols = [symbol],
    start = now - timedelta(days = 5),                          # specify start datetime, default=the beginning of the current day.
    # end=None,                                             # specify end datetime, default=now
    limit = 2,                                                # specify limit
)
crypto_historical_data_client.get_crypto_trades(req).df

In [ ]:
# get latest quotes by symbol
req = CryptoLatestQuoteRequest(
    symbol_or_symbols = [symbol],
)
res = crypto_historical_data_client.get_crypto_latest_quote(req)
res